## General Data Wrangling

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
data_full = pd.read_csv( '../01-data/data_merged_20220910_norm.csv')
data_full = data_full.query('H_Age<100&Draw>0').reset_index( drop = True)
y = data_full[ 'Winner']

C:\Users\User\AppData\Local\Temp\ipykernel_14460\2406342619.py:1: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  data_full = pd.read_csv( '../01-data/data_merged_20220910_norm.csv')


In [4]:
X = data_full.loc[ :, data_full.columns.str.startswith( 'ActualWeight') | data_full.columns.str.startswith( 'DeclaredHorseWeight') | \
    data_full.columns.str.startswith( 'WinOdds') | data_full.columns.str.startswith( 'Draw') | \
    data_full.columns.str.startswith( 'J_Total_') | data_full.columns.str.startswith( 'J_TotalStakes_') | \
    data_full.columns.str.startswith( 'J_Rate_') | \
    data_full.columns.str.startswith( 'T_Total_') | data_full.columns.str.startswith( 'T_TotalStakes_') | \
    data_full.columns.str.startswith( 'T_Rate') | \
    data_full.columns.str.startswith( 'H_Rate_') | data_full.columns.str.startswith( 'H_Total') | \
    data_full.columns.str.startswith( 'H_TotalStake') | data_full.columns.str.startswith( 'CombinedRating') | \
    data_full.columns.str.startswith( 'H_Age') | data_full.columns.str.startswith( 'WeightDiff') | \
    data_full.columns.str.startswith( 'LastRace_nDays') | data_full.columns.str.startswith( 'PreviousPlace') | \
    data_full.columns.str.startswith( 'AvgPlace3') | data_full.columns.str.startswith( 'AvgPlace5') | \
    data_full.columns.str.startswith( 'Place')] # Place is added for downsampling

print( X.columns)

Index(['Place', 'ActualWeight', 'DeclaredHorseWeight', 'Draw', 'WinOdds',
       'J_Rate_win', 'J_Rate_place', 'J_Rate_show', 'T_Rate_win',
       'T_Rate_place', 'T_Rate_show', 'H_TotalStake', 'H_Total', 'H_Rate_win',
       'H_Rate_place', 'H_Rate_show', 'H_Age', 'LastRace_nDays',
       'PreviousPlace', 'AvgPlace3', 'AvgPlace5', 'WeightDiff',
       'CombinedRating', 'Place_norm', 'ActualWeight_norm',
       'DeclaredHorseWeight_norm', 'Draw_norm', 'WinOdds_norm',
       'J_Rate_win_norm', 'J_Rate_place_norm', 'J_Rate_show_norm',
       'T_Rate_win_norm', 'T_Rate_place_norm', 'T_Rate_show_norm',
       'H_TotalStake_norm', 'H_Total_norm', 'H_Rate_win_norm',
       'H_Rate_place_norm', 'H_Rate_show_norm', 'H_Age_norm',
       'LastRace_nDays_norm', 'PreviousPlace_norm', 'AvgPlace3_norm',
       'AvgPlace5_norm', 'WeightDiff_norm', 'CombinedRating_norm'],
      dtype='object')


In [5]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.8, random_state = 918)

Variables and corresponding transformations. (All relative indexes will not be transformed as they are theoretically normal)


| Variable | Log? | Scaling |
|----------|------|---------|
| ActualWeight | No | Standard |
| DeclaredHorseWeight | No | Standard |
| Draw | No | Standard (_Ordinal_+Scaling) |
| WinOdds | Yes  |Standard |
| J_Rate_* | No | None |
| T_Rate_* | No | None |
| H_TotalStake | Yes | Standard |
| H_Total | Yes | Standard |
| H_Rate_* | No | None |
| H_Age | No | Standard |
| LastRace_nDays | Yes | Standard |
| PreviousPlace | No | Standard |
| AvgPlace* | No | Standard |
| WeightDiff | No | Standard |
| Rating | No | Standard |


In [6]:
cols_std = [
    'ActualWeight', 'DeclaredHorseWeight', 'Draw', 'H_Age', 'PreviousPlace', 'AvgPlace3', 'AvgPlace5', 'WeightDiff', 'CombinedRating']

cols_log_std = [
    'WinOdds', 'H_TotalStake', 'H_Total', 'LastRace_nDays']

cols_passthrough = [
    'J_Rate_win', 'J_Rate_place', 'J_Rate_show', 'T_Rate_win', 'T_Rate_place', 'T_Rate_show',
    'H_Rate_win', 'H_Rate_place', 'H_Rate_show',
    'ActualWeight_norm', 'DeclaredHorseWeight_norm', 'WinOdds_norm',
    'J_Rate_win_norm', 'J_Rate_place_norm', 'J_Rate_show_norm', 'T_Rate_win_norm', 'T_Rate_place_norm', 'T_Rate_show_norm',
    'H_TotalStake_norm', 'H_Total_norm', 'H_Rate_win_norm', 'H_Rate_place_norm', 'H_Rate_show_norm', 'H_Age_norm',
    'LastRace_nDays_norm', 'PreviousPlace_norm', 'AvgPlace3_norm', 'AvgPlace5_norm', 'WeightDiff_norm', 'CombinedRating_norm']

cols_drop = [
    'Place', 'Place_norm', 'Draw_norm']

In [7]:
def log_func(x):
    return np.log(x+1)

In [8]:
log_transformer = FunctionTransformer( log_func)

In [9]:
pipe_log_std = make_pipeline(
    log_transformer, StandardScaler()
)

In [10]:
column_transformer = make_column_transformer(
    ( StandardScaler(), cols_std),
    ( pipe_log_std, cols_log_std),
    ( 'passthrough', cols_passthrough),
    ( 'drop', cols_drop)
)

## KNN

In [11]:
pipe_knn = make_pipeline( column_transformer, KNeighborsClassifier())

In [16]:
cv_result = cross_validate( pipe_knn, X_train, y_train, cv = 5, return_train_score = True, scoring = 'precision')

In [17]:
pd.DataFrame( cv_result)

,fit_time,score_time,test_score,train_score
0,0.026145,0.136079,0.352941,0.660714
1,0.014037,0.085500,0.250000,0.742424
2,0.015023,0.086083,0.428571,0.661290
3,0.016983,0.085073,0.352941,0.754098
4,0.022173,0.135121,0.312500,0.666667


In [18]:
pd.DataFrame( cv_result).mean()

fit_time       0.018872
score_time     0.105571
test_score     0.339391
train_score    0.697039
dtype: float64

## Hyperparameter optimization

In [26]:
param_grid = {
    "kneighborsclassifier__n_neighbors": list( range( 5, 35, 5))
}

grid_search = GridSearchCV(
    pipe_knn, param_grid, cv = 5, scoring = 'precision', n_jobs=-1, return_train_score = True
)

In [27]:
grid_search.fit( X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['ActualWeight',
                                                                          'DeclaredHorseWeight',
                                                                          'Draw',
                                                                          'H_Age',
                                                                          'PreviousPlace',
                                                                          'AvgPlace3',
                                                                          'AvgPlace5',
                                                                          'WeightDiff',
                                                                          'CombinedRating']),
                                                                        ('pipeline',
                                                                         Pipeline(steps=[('functiontransformer',
                                                                                          FunctionTransformer(func=<function log_func...
                                                                          'H_Age_norm',
                                                                          'LastRace_nDays_norm',
                                                                          'PreviousPlace_norm',
                                                                          'AvgPlace3_norm',
                                                                          'AvgPlace5_norm',
                                                                          'WeightDiff_norm',
                                                                          'CombinedRating_norm']),
                                                                        ('drop',
                                                                         'drop',
                                                                         ['Place',
                                                                          'Place_norm',
                                                                          'Draw_norm'])])),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'kneighborsclassifier__n_neighbors': [5, 10, 15, 20,
                                                               25, 30]},
             return_train_score=True, scoring='precision')

In [28]:
results = pd.DataFrame(grid_search.cv_results_)
results[ ['params', 'mean_test_score']]

,params,mean_test_score
0,{'kneighborsclassifier__n_neighbors': 5},0.339391
1,{'kneighborsclassifier__n_neighbors': 10},0.433333
2,{'kneighborsclassifier__n_neighbors': 15},0.391111
3,{'kneighborsclassifier__n_neighbors': 20},0.633333
4,{'kneighborsclassifier__n_neighbors': 25},0.644286
5,{'kneighborsclassifier__n_neighbors': 30},0.566667


## Final model evaluation

In [29]:
pipe_knn_opt = make_pipeline( column_transformer, KNeighborsClassifier( n_neighbors = 20))

In [30]:
pipe_knn_opt.fit( X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['ActualWeight',
                                                   'DeclaredHorseWeight',
                                                   'Draw', 'H_Age',
                                                   'PreviousPlace', 'AvgPlace3',
                                                   'AvgPlace5', 'WeightDiff',
                                                   'CombinedRating']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('functiontransformer',
                                                                   FunctionTransformer(func=<function log_func at 0x0000020C2B006820>)),
                                                                  ('st...
                                                   'T_Rate_show_norm',
                                                   'H_TotalStake_norm',
                                                   'H_Total_norm',
                                                   'H_Rate_win_norm',
                                                   'H_Rate_place_norm',
                                                   'H_Rate_show_norm',
                                                   'H_Age_norm',
                                                   'LastRace_nDays_norm',
                                                   'PreviousPlace_norm',
                                                   'AvgPlace3_norm',
                                                   'AvgPlace5_norm',
                                                   'WeightDiff_norm',
                                                   'CombinedRating_norm']),
                                                 ('drop', 'drop',
                                                  ['Place', 'Place_norm',
                                                   'Draw_norm'])])),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=20))])

In [31]:
y_hat = pipe_knn_opt.predict( X_test)

In [32]:
print( confusion_matrix( y_test, y_hat, labels = [ True, False]))
print( classification_report( y_test, y_hat))

[[   40  1044]
 [   34 11997]]
              precision    recall  f1-score   support

       False       0.92      1.00      0.96     12031
        True       0.54      0.04      0.07      1084

    accuracy                           0.92     13115
   macro avg       0.73      0.52      0.51     13115
weighted avg       0.89      0.92      0.88     13115

